# Demo 1: Fallzahlen, Genesungen, und Todesfällen Vergleichen

## Importing Dependencies

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as wgts
from IPython.display import display

## Loading Datasets

In [4]:
# test data read
covid_19_dataset = pd.read_csv('../dataset/time-series-19-covid-combined.csv')
regions_dataset = pd.read_csv('../dataset/Countries by continents.csv')

## Creating Countries & Continents Dropdown Menus

In [ ]:
countries_dropdown_menu = wgts.Dropdown(
    options= regions_dataset['Country'].unique(),
    description='Country:',
    disabled=False,
)

continent_dropdown_menu = wgts.Dropdown(
    options= regions_dataset['Continent'].unique(),
    description='Continent:',
    disabled=False,
)
wgts.HBox([continent_dropdown_menu, countries_dropdown_menu])

TypeError: Box.__init__() takes from 1 to 2 positional arguments but 3 were given